In [1]:
# stdlib
import os

# syft absolute
import syft as sy
from syft.client.syncing import compare_states
from syft.client.syncing import resolve_single

In [2]:
node_low = sy.orchestra.launch(
    name="reddit_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="reddit_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=dfda254715f34ba28c4d91acde7d8ff7 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=9bf6a64a9252480abe50017d5cf5ab32 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <reddit_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <reddit_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

SyftSuccess: User 'John Doe' successfully registered! To see users, run `[your_client].users`

In [5]:
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <reddit_l: Low side Domain> as <newuser@openmined.org>


# Setup twin api

In [6]:
BQ_PRIVATE_KEY = os.environ["BQ_PRIVATE_KEY"].replace("\\n", "\n")

In [7]:
bq_credentials = {
    "type": "service_account",
    "project_id": "reddit-testing-415005",
    "private_key_id": "0b46406c4b049257f4c6f8f2473a2a05bf3e5321",
    "private_key": BQ_PRIVATE_KEY,
    "client_email": "rdt-testing-sv-acc@reddit-testing-415005.iam.gserviceaccount.com",
    "client_id": "118222186980956490094",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/rdt-testing-sv-acc%40reddit-testing-415005.iam.gserviceaccount.com",
    "universe_domain": "googleapis.com",
}

In [8]:
# Mock Behavior


@sy.mock_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def mock_query_function(
    context,  # Variable used to track user session, user information, user activities and settings
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )

    # Replacing private values to mocked ones.
    result = rows.to_dataframe()
    result["int64_field_0"] = 0
    result["id"] = "Private"
    result["name"] = "Private"
    result["subscribers_count"] = 0
    result["permalink"] = "Private"
    result["nsfw"] = "NaN"
    result["spam"] = False
    return result

In [9]:
# Private Behavior


@sy.private_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def private_query_function(
    context,
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    print("test")
    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )
    print(client)

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )
    print(rows)

    return rows.to_dataframe()

In [10]:
new_endpoint = sy.TwinAPIEndpoint(
    path="reddit.query",
    private_function=private_query_function,
    mock_function=mock_query_function,
    description="Lorem ipsum dolor sit amet lorem adipiscing elit …",
)
client_high.api.services.api.add(endpoint=new_endpoint)

SyftSuccess: Endpoint successfully created.

In [11]:
client_high.refresh()

In [12]:
client_high.api.services.reddit.query.private(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

test


,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


In [13]:
twin_api_obj = client_high.api.services.api.api_endpoints()[0]

## Sync twin api

In [14]:
#CustomEndpointActionObject

In [15]:
state = client_high.get_sync_state()

In [16]:
state

syft.service.sync.sync_state.SyncState

In [14]:
twin_api_obj = (
    node_high.python_node.service_path_map["apiservice"]
    .stash.get_all(client_high.credentials)
    .ok()[0]
)

In [21]:
custom_actions = list(
    node_high.python_node.service_path_map["actionservice"].store.data.values()
)
custom_action = custom_actions[0]

In [25]:
isinstance(custom_action, CustomEndpointActionObject)

NameError: name 'CustomEndpointActionObject' is not defined

In [ ]:
node_high.python_node.get_service("actionservic")

What we need to write

In [16]:
node_low.python_node.service_path_map["apiservice"].stash.upsert(
    credentials=client_low.credentials, endpoint=twin_api_obj
)

Ok(syft.service.api.api.TwinAPIEndpoint)

In [17]:
client_low.api.services.action.set(custom_action)

```python
Pointer
```
```python
class CustomEndpointActionObject:
  id: str = 891832e0056c4273b408323057a43684

```

# Create request

## Use mock endpoint

In [18]:
client_low.refresh()
client_low_ds.refresh()

In [19]:
# client_low.api.services.api.api_endpoints()

In [20]:
# client_low.api.services.reddit.query.mock(sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100")

In [21]:
client_low_ds.api.services.reddit.query(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,0,Private,Private,0,Private,NaN,False
1,0,Private,Private,0,Private,NaN,False
2,0,Private,Private,0,Private,NaN,False
3,0,Private,Private,0,Private,NaN,False
4,0,Private,Private,0,Private,NaN,False
...,...,...,...,...,...,...,...
95,0,Private,Private,0,Private,NaN,False
96,0,Private,Private,0,Private,NaN,False
97,0,Private,Private,0,Private,NaN,False
98,0,Private,Private,0,Private,NaN,False


# Define code, project, request

In [22]:
@sy.syft_function_single_use(
    reddit_query=client_low_ds.api.services.reddit.query,
)
def my_research_pipeline(reddit_query):
    sql_query = "SELECT *  FROM test_1gb.subreddits LIMIT 100"
    return reddit_query(sql_query=sql_query)

SyftSuccess: Syft function 'my_research_pipeline' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [23]:
new_project = sy.Project(
    name="Reddit Research Studies",
    description="Hi, I want to get information about your data.",
    members=[client_low_ds],
)

new_project.create_code_request(my_research_pipeline, client_low_ds)

SyftSuccess: Code request for 'my_research_pipeline' successfully added to 'Reddit Research Studies' Project. To see code requests by a client, run `[your_client].code`

In [24]:
# low_request = client_low.requests[-1]

In [25]:
# low_request

In [26]:
# low_code = low_request.code

In [27]:
# low_code

## Sync code request to high side

In [28]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [29]:
low_state

syft.service.sync.sync_state.SyncState

In [30]:
# syft absolute

In [31]:
diff_state = compare_states(low_state, high_state)

In [32]:
diff_state

syft.service.sync.diff_state.NodeDiff

### Sync UserCode

In [33]:
obj_diff_batch = diff_state[0]

In [34]:
widget = resolve_single(obj_diff_batch)

In [35]:
widget

In [ ]:
widget.click_sync()

### Sync Request

In [ ]:
obj_diff_batch_request = diff_state[1]

In [ ]:
widget = resolve_single(obj_diff_batch_request)

In [ ]:
widget

In [ ]:
widget.click_sync()

# High side: Run and sync back

## Run on high side

In [ ]:
client_high.refresh()

In [47]:
client_high.requests

In [48]:
request = client_high.requests[0]

In [52]:
# request.code

In [53]:
res = client_high.code.my_research_pipeline(
    reddit_query=client_high.api.services.reddit.query
)

test


In [54]:
res

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


In [55]:
request.accept_by_depositing_result(res)

Approving request for domain reddit_h
Creating job for existing user code
Approving request for domain reddit_h
ADDING PERMISSION [READ: 6de05d6a39674d73853dbe2b6701f468 as 3bb15e0cb43fb82513ab6984ae0a60915c0353e0e940f3397131f28f010df928] 6de05d6a39674d73853dbe2b6701f468
Job(98fac4841a9f4ca5a135e39b2fc59472) Setting new result None -> 6de05d6a39674d73853dbe2b6701f468


SyftSuccess: Request submitted for updating result.

## Sync back to low side

In [56]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [57]:
high_state

syft.service.sync.sync_state.SyncState

In [58]:
diff_state_2 = compare_states(high_state, low_state)

In [59]:
diff_state_2

syft.service.sync.diff_state.NodeDiff

In [62]:
code_batch = diff_state_2[0]
# code_batch

In [63]:
widget = resolve_single(code_batch)
widget

In [64]:
res = widget.click_sync()

Decision: Syncing 2 objects


In [65]:
request_batch = diff_state_2[1]

In [66]:
request_batch

LOW SIDE STATE:

―――― MODIFIED ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #f31263...
changes: syft.service.request.request.UserCodeStatusChange
history: [
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #743393...
    service_func_name: my_research_pipeline
    input_owners: reddit_l
    code_status: Node: reddit_l, Status: approved



HIGH SIDE STATE:

―――― MODIFIED ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #f31263...
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange
]
history: [
‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus
‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus
‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #743393...
    service_func_name: my_research_pipeline
    input_owners: reddit_l
    code_status: Node: reddit_h, Status: approved

In [67]:
widget = resolve_single(request_batch)
widget

In [68]:
# res = widget.click_share_all_private_data()

In [69]:
res = widget.click_sync()

Decision: Syncing 1 objects


In [70]:
res

SyftSuccess: Synced 1 items

In [71]:
job_batch = diff_state_2[2]

In [73]:
# job_batch

In [74]:
widget = resolve_single(job_batch)
widget

In [75]:
widget.click_share_all_private_data()
widget.click_sync()

Decision: Syncing 4 objects


SyftSuccess: Synced 4 items

In [76]:
# client_low.get_sync_state()

In [77]:
# %debug

# Run on low side

In [78]:
client_low_ds.refresh()
client_low_ds.code.my_research_pipeline(
    reddit_query=client_low_ds.api.services.reddit.query
)

SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False
